In [30]:
import warnings

warnings.filterwarnings("ignore")

import os
import sys
import time

import pandas as pd
import numpy as np

## Part a: Extracting Language Features

In [31]:
# the interview data (sheet 1)
interview_data = pd.read_excel(
    "../data/DAIC_demographic_data.xlsx",
    sheet_name="Interview_Data",
    skiprows=lambda x: x == 1,
)
# drop the rows where data in column Partic# is NaN
interview_data = interview_data.dropna(subset=["Partic#"])
print(interview_data.tail())

# the phq score data (sheet 2)
phq_score = pd.read_excel(
    "../data/DAIC_demographic_data.xlsx", sheet_name="Metadata_mapping"
)

print(phq_score.head())

     Partic# Condition  gender  race
428    837.0        AI     2.0     3
429    838.0        AI     1.0     1
430    839.0        AI     1.0     1
431    840.0        AI     2.0     3
432    841.0        AI     1.0     4
   Participant_ID  PHQ_Score
0             300          2
1             301          3
2             302          4
3             303          0
4             304          6


The following step was a workaround to prevent the end of file error due to unclosed inverted commas. What the code is doing - opening and reading each file, reading each line and checking for lines that start with " but does not end with " (i.e., checking for unclosed quotes).

In [32]:
# preprocessing the csv files to remove the unclosed inverted commas
def clean_csv(file_path, output_path):
    with open(file_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            if line.startswith('"') and not line.endswith('"\n'):
                line = line[1:]
            outfile.write(line)


def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".csv"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            clean_csv(input_path, output_path)


input_dir = "../data/E-DAIC_Transcripts"
output_dir = "../data/E-DAIC_Transcripts_cleaned"

process_directory(input_dir, output_dir)

In [33]:
cleaned_transcripts_path = "../data/E-DAIC_Transcripts_cleaned"

transcripts = {}

# loop through each file in the folder, load it, and store the content
for filename in os.listdir(cleaned_transcripts_path):
    if filename.endswith("_Transcript.csv"):
        participant_id = filename.split("_")[0]  # extract the participant id
        file_path = os.path.join(cleaned_transcripts_path, filename)

        df = pd.read_csv(file_path)
        transcripts[participant_id] = df

# transcripts
transcripts["386"]

,Text
0,might have pulled something that
1,I'm going to bring the great thanks so much
2,and please
3,are you okay with this yes
4,oh I'm fine I'm a little tired but I found ou...
...,...
76,yeah well after college people usually many p...
77,thank you goodbye
78,oh that was that was fast
79,but I didn't never said there wasn't any like...


In [34]:
# combine all the text data for each participant into a single string
combined_transcripts = {}

for participant_id, transcript in transcripts.items():
    combined_transcripts[participant_id] = " ".join(transcript["Text"].astype(str))

# removing extra spaces caused by newlines
for participant_id, transcript in combined_transcripts.items():
    combined_transcripts[participant_id] = " ".join(transcript.split())

# combined_transcripts["386"]

In [35]:
# combine the demographic data with the extracted text data

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# create the feature extractor objects here; using the base settings for now
# TODO: make changes to the parameters to preprocess the text data

corpus = list(combined_transcripts.values())
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(corpus)

count_vectorizer = CountVectorizer()
count_vectorizer.fit(corpus)

sentiment_analyzer = SentimentIntensityAnalyzer()

combined_data = []

# convert the participant id in demographic data to int for consistency
interview_data["Partic#"] = interview_data["Partic#"].astype(int)

for _, row in interview_data.iterrows():
    participant_id = str(row["Partic#"])  # convert id to match the transcript ids

    # find langauge features for this participant
    if participant_id in combined_transcripts:
        transcript = combined_transcripts[participant_id]

        # tfidf features
        tfidf_features = (
            tfidf_vectorizer.transform([transcript]).toarray().flatten()
        )  # has to be 1D array

        # count features
        count_features = count_vectorizer.transform([transcript]).toarray().flatten()

        # sentiment features
        sentiment_compound_scores = sentiment_analyzer.polarity_scores(transcript)[
            "compound"
        ]  # only extracting the compound score

        # combine all the features
        data = {
            "participant_id": participant_id,
            "condition": row["Condition"],
            "race": row["race"],
            "gender": row["gender"],
            "tfidf_features": tfidf_features,
            "count_features": count_features,
            "overall_sentiment": sentiment_compound_scores,
        }

        combined_data.append(data)

# convert the combined data into a dataframe
combined_data_df = pd.DataFrame(combined_data)

# print(combined_data_df.head())

In [36]:
combined_data_df.head()

,participant_id,condition,race,gender,tfidf_features,count_features,overall_sentiment
0,386,WoZ,3,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0085443313426...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.9999
1,387,WoZ,1,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0253774451976...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.9996
2,388,WoZ,4,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0292778521617...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.9953
3,389,WoZ,1,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0487055979434...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, ...",0.9822
4,390,WoZ,3,1.0,"[0.03182282795453763, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.9996


Some useful information about the resulting dataframe - 

- Each term in the `TF-IDF` vector is considered a feature. The values represent the `TF-IDF` score for that term. A higher score could mean that the term is important to the transcript but no common in the entire list of transcripts. This is helpful in identifying the transcript's topic/sentiment.
- Count features are straightforward. The value of a feature is the raw count of how many times the term appears in the transcript.
- The compound score is the overall sentiment of the transcript. Its value ranges from $-1$ to $1$ where positive sentiments have a compound score of $\geq 0.05$, neutral sentiment have a score between $-0.05$ and $0.05$, and negative sentiments have a compound score of $\leq -0.05$.

## Part b: Classifying for gender

In [11]:
# drop race, condition, and participant_id
genderDF = combined_data_df.drop(["race", "condition", "participant_id"], axis=1)

# drop any row with NaN
genderDF = genderDF.dropna()

# map gender from [1,2] -> [0,1] (XGBoost needs the labels to be 0 or 1)
genderDF["gender"] = genderDF["gender"].map({1: 0, 2: 1})

# extract the target values (gender)
y = genderDF.pop("gender")

In [12]:
# XGBoost cannot accept multi-dimensional features so each list element must live in its own column

# Expand lists into separate columns
tfidf_features = pd.DataFrame(genderDF['tfidf_features'].tolist(), index=genderDF.index)
tfidf_features.columns = [f'tfidf_features{i}' for i in range(tfidf_features.shape[1])]

# Expand lists into separate columns
count_features = pd.DataFrame(genderDF['count_features'].tolist(), index=genderDF.index)
count_features.columns = [f'count_features{i}' for i in range(count_features.shape[1])]

# recreate gender dataframe
genderDF = pd.concat([tfidf_features, count_features], axis=1)

genderDF.shape

(188, 17404)

In [13]:
# due to the flattening of the arrays, we not how 17404 features which is too much
# therefore, we are going to use PCA to reduce our input dimensionality

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Scale data before applying PCA
scaling=StandardScaler()
 
# Use fit and transform method 
scaling.fit(genderDF)
Scaled_data=scaling.transform(genderDF)

In [14]:
# helper function to calcualte accuracy and balanced accuracy
def getAccAndBAcc(yPred, yTrue):

    truePos = 0
    trueNeg = 0
    falsePos = 0
    falseNeg = 0

    for idx in range(len(yPred)):
        
        if yPred[idx] == 1:

            if yTrue[idx] == 1:

                truePos += 1

            else:

                falseNeg += 1

        else:

            if yTrue[idx] == 1:

                falsePos += 1

            else:

                trueNeg += 1

    if (trueNeg+falsePos) == 0 and (truePos+falseNeg) != 0:
        balancedAccuracy = 0.5*truePos/(truePos+falseNeg)
    elif (trueNeg+falsePos) != 0 and (truePos+falseNeg) == 0:
        balancedAccuracy = 0.5*trueNeg/(trueNeg+falsePos)
    else:
        balancedAccuracy = 0.5*trueNeg/(trueNeg+falsePos) + 0.5*truePos/(truePos+falseNeg)
    
    accuracy = (truePos + trueNeg) / (truePos + trueNeg + falsePos + falseNeg)

    return accuracy, balancedAccuracy

In [15]:
# create deep learning model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def getDLModel(inputShape):

    model = Sequential([
        Dense(128, activation='relu', input_shape=(inputShape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics = ["accuracy"]
    )

    return model

ModuleNotFoundError: No module named 'tensorflow.keras'

In [224]:
from sklearn.model_selection import train_test_split
#!pip install xgboost
import xgboost as xgb

# minimum covariance for feature to be included
pcaCovarianceFloor = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

bAccTree = []
accTree  = []

bAccDL = []
accDL  = []

numberOfComponents = []

for pcaFloor in pcaCovarianceFloor:

    # run PCA with given number of components
    principal=PCA(n_components=pcaFloor)
    principal.fit(Scaled_data)
    x=principal.transform(Scaled_data)

    # get the reduced dataset
    reducedGenderData = principal.transform(Scaled_data)
    numberOfComponents.append(reducedGenderData.shape[1])
    reducedGenderDF = pd.DataFrame(reducedGenderData, columns=[f"PC{i}" for i in range(1, numberOfComponents[-1] + 1)])

    # split the dataset into testing and training splits
    # since the dataset is so small, we want to make sure 
    # there is at least 10 of each class in test split
    xTrain, xTest, yTrain, yTest = train_test_split(reducedGenderDF, y, test_size=0.2, random_state=76)

    # train XGBoost model
    xgb_classifier = xgb.XGBClassifier()
    xgb_classifier.fit(xTrain, yTrain)

    # get test accuracy and balanced accuracy
    acc, bAcc = getAccAndBAcc(xgb_classifier.predict(xTest), yTest.to_numpy())

    # store accuracies
    accTree.append(acc)
    bAccTree.append(bAcc)

    # train DL model
    FNN = getDLModel(numberOfComponents[-1])
    FNN.fit(xTrain, yTrain, epochs=100, verbose=0)

    # get test accuracy and balanced accuracy
    acc, bAcc = getAccAndBAcc(FNN.predict(xTest), yTest.to_numpy())

    # store accuracies
    accDL.append(acc)
    bAccDL.append(bAcc)

print("\n")
for idx in range(len(pcaCovarianceFloor)):

    print("#######################################")
    print(f"Values for Cov. Floor: {pcaCovarianceFloor[idx]}, Components: {numberOfComponents[idx]}:")
    print(f"Tree accuracy: {accTree[idx]}")
    print(f"Tree balanced accuracy: {bAccTree[idx]}")
    print(f"DL accuracy: {accDL[idx]}")
    print(f"DL balanced accuracy: {bAccDL[idx]}")


maxTreeBAccPCACovFloor = pcaCovarianceFloor[np.argmax(bAccTree)]
maxDLBAccPCACovFloor = pcaCovarianceFloor[np.argmax(bAccDL)]

print("\n")
print("#######################################")
print(f"Max Tree Covariance Floor: {maxTreeBAccPCACovFloor}, with balanced accuracy of {np.max(bAccTree) * 100}%")
print(f"Max DL Covariance Floor: {maxDLBAccPCACovFloor}, with balanced accuracy of {np.max(bAccDL) * 100}%")
print("#######################################")

2/2 [==============================] - 0s 0s/step


#######################################
Values for Cov. Floor: 0.1, Components: 8:
Tree accuracy: 0.631578947368421
Tree balanced accuracy: 0.606060606060606
DL accuracy: 0.631578947368421
DL balanced accuracy: 0.8108108108108107
#######################################
Values for Cov. Floor: 0.2, Components: 18:
Tree accuracy: 0.6578947368421053
Tree balanced accuracy: 0.6378205128205128
DL accuracy: 0.7105263157894737
DL balanced accuracy: 0.7604166666666667
#######################################
Values for Cov. Floor: 0.3, Components: 30:
Tree accuracy: 0.7105263157894737
Tree balanced accuracy: 0.7071428571428571
DL accuracy: 0.631578947368421
DL balanced accuracy: 0.6476190476190475
#######################################
Values for Cov. Floor: 0.4, Components: 43:
Tree accuracy: 0.7105263157894737
Tree balanced accuracy: 0.7071428571428571
DL accuracy: 0.6052631578947368
DL balanced accuracy: 0.3026315789473684
#################

# part c

In [76]:
# imports
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, BatchNormalization, Attention, Flatten, Input
from keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [37]:
def create_df(input_df, col_to_drop, y_col):
    new_df = input_df.drop(col_to_drop, axis=1)
    new_df = new_df.dropna()
    y_new = new_df.pop(y_col)
    tfidf_features = pd.DataFrame(new_df['tfidf_features'].tolist(), index=new_df.index)
    tfidf_features.columns = [f'tfidf_features{i}' for i in range(tfidf_features.shape[1])]

    count_features = pd.DataFrame(new_df['count_features'].tolist(), index=new_df.index)
    count_features.columns = [f'count_features{i}' for i in range(count_features.shape[1])]
    
    new_df = pd.concat([tfidf_features, count_features], axis=1)
    return new_df, y_new

In [45]:
race_df, y_race = create_df(combined_data_df, ["gender", "condition", "participant_id"], "race")
race_df.shape, y_race.shape

((190, 17404), (190,))

In [83]:
# xgboost needs the range to start at 0
y_race = y_race - 1
y_race

0      2
1      0
2      3
3      0
4      2
      ..
185    2
186    0
187    3
188    0
189    0
Name: race, Length: 190, dtype: int64

In [44]:
def scale_data(input_df):
    scaling=StandardScaler()
    scaling.fit(input_df)
    Scaled_data=scaling.transform(input_df)
    return Scaled_data

In [70]:
def get_dense_model(input_shape):
    
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        BatchNormalization(),
        Dropout(0.3),
    
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
    
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
    
        Dense(16, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
    
        Dense(8, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
    
        Dense(1, activation="sigmoid")
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [84]:
scaled_data = scale_data(race_df)

pcaCovarianceFloor = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

bA_dense, ba_xgb = [], []
a_dense, a_xgb = [], []

num_comp = []

for pcaFloor in pcaCovarianceFloor:
    
    principal=PCA(n_components=pcaFloor)
    principal.fit(scaled_data)
    x=principal.transform(scaled_data)
    
    reduced_race_df = principal.transform(scaled_data)
    num_comp.append(reduced_race_df.shape[1])
    reduced_race_df = pd.DataFrame(reduced_race_df, columns=[f"PC{i}" for i in range(1, num_comp[-1] + 1)])
    
    x_train, x_test, y_train, y_test = train_test_split(reduced_race_df, y_race, test_size=0.2, random_state=76)

    attention_model = get_dense_model(num_comp[-1])
    attention_model.fit(x_train, y_train, epochs=100, verbose=0)

    acc, bAcc = getAccAndBAcc(attention_model.predict(x_test), y_test.to_numpy())

    a_dense.append(acc)
    bA_dense.append(bAcc)
    
    xgb_classifier = xgb.XGBClassifier()
    xgb_classifier.fit(x_train, y_train)

    acc, bAcc = getAccAndBAcc(xgb_classifier.predict(x_test), y_test.to_numpy())
    
    a_xgb.append(acc)
    ba_xgb.append(bAcc)
    

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [85]:
print("\n")
for idx in range(len(pcaCovarianceFloor)):

    print("#######################################")
    print(f"Values for Cov. Floor: {pcaCovarianceFloor[idx]}, Components: {num_comp[idx]}:")
    print(f"Tree accuracy: {a_xgb[idx]}")
    print(f"Tree balanced accuracy: {ba_xgb[idx]}")
    print(f"DL accuracy: {a_dense[idx]}")
    print(f"DL balanced accuracy: {bA_dense[idx]}")


maxTreeBAccPCACovFloor = pcaCovarianceFloor[np.argmax(ba_xgb)]
maxDLBAccPCACovFloor = pcaCovarianceFloor[np.argmax(bA_dense)]

print("\n")
print("#######################################")
print(f"Max Tree Covariance Floor: {maxTreeBAccPCACovFloor}, with balanced accuracy of {np.max(ba_xgb) * 100}%")
print(f"Max DL Covariance Floor: {maxDLBAccPCACovFloor}, with balanced accuracy of {np.max(bA_dense) * 100}%")
print("#######################################")



#######################################
Values for Cov. Floor: 0.1, Components: 8:
Tree accuracy: 0.8947368421052632
Tree balanced accuracy: 0.4594594594594595
DL accuracy: 0.9210526315789473
DL balanced accuracy: 0.4605263157894737
#######################################
Values for Cov. Floor: 0.2, Components: 18:
Tree accuracy: 0.9210526315789473
Tree balanced accuracy: 0.4605263157894737
DL accuracy: 0.9210526315789473
DL balanced accuracy: 0.4605263157894737
#######################################
Values for Cov. Floor: 0.3, Components: 30:
Tree accuracy: 0.9210526315789473
Tree balanced accuracy: 0.4605263157894737
DL accuracy: 0.8947368421052632
DL balanced accuracy: 0.4594594594594595
#######################################
Values for Cov. Floor: 0.4, Components: 44:
Tree accuracy: 0.9210526315789473
Tree balanced accuracy: 0.4605263157894737
DL accuracy: 0.9210526315789473
DL balanced accuracy: 0.4605263157894737
#######################################
Values for Cov. Floor: 